                              ПОИСК ПРОПУСКОВ В СТОЛБЦАХ В ПРОЦЕНТАХ
                              

In [ ]:
cols_null_percent = sber_data.isnull().mean() * 100
cols_with_null = cols_null_percent[cols_null_percent>0].sort_values(ascending=False)
colors = ['blue', 'yellow'] #Теплова карта для поиска пропусков
fig = plt.figure(figsize=(10, 4))
cols = cols_with_null.index
ax = sns.heatmap(
    sber_data[cols].isnull(),
    cmap=sns.color_palette(colors),
)
----
cols = cols_with_null.index # гистограммы с пропусками
sber_data[cols].hist(figsize=(20, 8));

                         УДАЛЕНИЕ СТРОК И СТОЛБЦОВ ПО ПРОПУСКАМ

In [ ]:
#задаем минимальный порог: вычисляем 70% от числа строк
thresh = drop_data.shape[0]*0.7
#удаляем столбцы, в которых более 30% (100-70) пропусков
drop_data = drop_data.dropna(thresh=thresh, axis=1)
#удаляем записи, в которых есть хотя бы 1 пропуск
drop_data = drop_data.dropna(how='any', axis=0)# how="all" - удалить если нет всех значений, any -хотя бы 1
#удаляем строки с пропусками в приведенных клонках
data = data.dropna(subset=['Последнее/нынешнее место работы', 'Последняя/нынешняя должность'])


                        ЗАМЕНА ПРОПУСКОВ ЗНАЧЕНИЯМИ ПО СЛОВАРЮ

In [ ]:
#создаём словарь 'имя_столбца': число (признак), на который надо заменить пропуски 
values = {
    'life_sq': combine_data['full_sq'],
    'metro_min_walk': combine_data['metro_min_walk'].median(),
    'metro_km_walk': combine_data['metro_km_walk'].median(),
    'railroad_station_walk_km': combine_data['railroad_station_walk_km'].median(),
    'railroad_station_walk_min': combine_data['railroad_station_walk_min'].median(),
    'preschool_quota': combine_data['preschool_quota'].mode()[0],
    'school_quota': combine_data['school_quota'].mode()[0],
    'floor': combine_data['floor'].mode()[0]
}
#заполняем оставшиеся записи константами в соответствии со словарем values
combine_data = combine_data.fillna(values)

                         ФУНКЦИЯ МЕТОДА ТЬЮКИ

In [ ]:
def outliers_iqr(data, feature,log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 3)
    upper_bound = quartile_3 + (iqr * 3)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')

                                 МЕТОД Z-ОТКЛОНЕНИЙ (МЕТОД СИГМ)

In [ ]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature]+1)
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned
outliers, cleaned = outliers_z_score(sber_data, 'mkad_km', log_scale=True)
print(f'Число выбросов по методу Z-отклонений: {outliers.shape[0]}')
print(f'Результирующее число записей: {cleaned.shape[0]}')


                                  ПОИСК ДУБЛИКАТОВ

In [ ]:
dupl_columns = list(sber_data.columns)
dupl_columns.remove('id')

mask = sber_data.duplicated(subset=dupl_columns)
sber_duplicates = sber_data[mask]
sber_dedupped = sber_data.drop_duplicates(subset=dupl_columns) # удаление дубликатов

                             СОЗДАЕМ СПИСОК НЕИНФОРМАТИВНЫХ ПРИЗНАКОВ
                        

In [ ]:
#список неинформативных признаков
low_information_cols = [] 

#цикл по всем столбцам
for col in sber_data.columns:
    #наибольшая относительная частота в признаке
    top_freq = sber_data[col].value_counts(normalize=True).max()
    #доля уникальных значений от размера признака
    nunique_ratio = sber_data[col].nunique() / sber_data[col].count()
    # сравниваем наибольшую частоту с порогом
    if top_freq > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(top_freq*100, 2)}% одинаковых значений')
    # сравниваем долю уникальных значений с порогом
    if nunique_ratio > 0.95:
        low_information_cols.append(col)
        print(f'{col}: {round(nunique_ratio*100, 2)}% уникальных значений')
information_sber_data = sber_data.drop(low_information_cols, axis=1)        

                               ПОДГРУЗКА EXCEL ФАЙЛОВ ПО СТРАНИЦАМ

In [ ]:
sheet1=pd.read_excel("Data_TSUM.xlsx",sheet_name="name")